# Get the Weather Description and Amount of Precipitation for Each City

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)

622

In [6]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=4207db3a377792eb24c750378dd88d03


In [9]:
# Create an empty list to hold the weather data.
city_data = []

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    city_url = url + "&q=" + city

  
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Parse out the needed data, adding dictionary index since it is nested.
        city_weather_description = city_weather["weather"][0]["description"]
        
        # Get the amount of rainfall in inches for the last 3 hours
        try:
            city_rain_inches = city_weather["rain"]["3h"] / 25.4
        
        except KeyError:
            city_rain_inches = 0
            
        # Get the amount of snowfall in inches for the last 3 hours
        try:
            city_snow_inches = city_weather["snow"]["3h"] / 25.4
        
        except KeyError:
            city_snow_inches = 0
                
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Convert rainfall and snowfall to inches
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description,
                          "Rain inches (last 3 hrs)": city_rain_inches,
                          "Snow inches (last 3 hrs)": city_snow_inches})

    # If an error is experienced, skip the city.
    except:
        pass

In [11]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,Bluff,NZ,2020-04-02 19:48:29,-46.60,168.33,53.01,95,53,3.00,broken clouds,0.000000,0.0
1,Khani,GE,2020-04-02 19:49:46,41.96,42.96,42.80,93,75,6.93,mist,0.000000,0.0
2,Alyangula,AU,2020-04-02 19:49:46,-13.85,136.42,73.40,100,30,3.36,scattered clouds,0.000000,0.0
3,Saldanha,ZA,2020-04-02 19:49:46,-33.01,17.94,75.20,33,0,8.05,clear sky,0.000000,0.0
4,Fukue,JP,2020-04-02 19:49:47,32.69,128.84,57.13,53,100,9.64,overcast clouds,0.000000,0.0
5,Torbay,CA,2020-04-02 19:48:28,47.67,-52.73,41.00,100,90,16.11,fog,0.000000,0.0
6,Tuktoyaktuk,CA,2020-04-02 19:48:25,69.45,-133.04,3.20,65,5,25.28,clear sky,0.000000,0.0
7,Bolobo,CD,2020-04-02 19:49:48,-2.17,16.23,77.72,77,54,4.99,light rain,0.054331,0.0
8,Aberdeen,GB,2020-04-02 19:49:48,57.14,-2.10,39.99,48,0,12.75,clear sky,0.000000,0.0
9,Aitape,PG,2020-04-02 19:48:35,-3.14,142.35,79.07,85,94,14.20,light rain,0.081496,0.0


In [12]:
# Create the output file (CSV).
output_challenge_data_file = "weather_data/WeatherPy_challenge.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_challenge_data_file)

In [23]:
# Create DataFrame.
cities_rain_df = city_data_df.loc[(city_data_df["Rain inches (last 3 hrs)"] > 0)]
cities_snow_df = city_data_df.loc[(city_data_df["Snow inches (last 3 hrs)"] > 0)]

In [32]:
cities_rain_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
7,Bolobo,CD,2020-04-02 19:49:48,-2.17,16.23,77.72,77,54,4.99,light rain,0.054331,0.0
9,Aitape,PG,2020-04-02 19:48:35,-3.14,142.35,79.07,85,94,14.20,light rain,0.081496,0.0
11,Cidreira,BR,2020-04-02 19:49:20,-30.18,-50.21,73.31,63,33,20.60,light rain,0.011417,0.0
17,Rikitea,PF,2020-04-02 19:48:20,-23.12,-134.97,77.05,73,85,6.73,light rain,0.027165,0.0
24,Kavieng,PG,2020-04-02 19:48:56,-2.57,150.80,82.83,75,15,5.41,light rain,0.012598,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
527,Pavilosta,LV,2020-04-02 19:51:36,56.89,21.19,44.60,75,100,28.86,light rain,0.016142,0.0
544,Mbanza-Ngungu,CD,2020-04-02 19:51:39,-5.25,14.87,72.77,82,100,3.80,light rain,0.029921,0.0
557,Canutama,BR,2020-04-02 19:48:04,-6.53,-64.38,88.07,64,60,1.21,light rain,0.030315,0.0
561,Mwene-Ditu,CD,2020-04-02 19:51:41,-7.00,23.45,68.76,75,100,6.20,light rain,0.100787,0.0


In [33]:
cities_snow_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
53,Severo-Kurilsk,RU,2020-04-02 19:49:04,50.68,156.12,32.09,95,100,42.28,snow,0.0,0.118504
58,Kristinehamn,SE,2020-04-02 19:50:00,59.31,14.11,37.99,85,93,13.87,light snow,0.0,0.014173
140,Klaksvik,FO,2020-04-02 19:48:44,62.23,-6.59,30.20,80,98,16.11,light snow,0.0,0.006693
251,Dikson,RU,2020-04-02 19:48:28,73.51,80.55,21.42,96,100,27.29,light snow,0.0,0.012992
312,Novoagansk,RU,2020-04-02 19:51:05,61.94,76.66,26.98,97,100,20.24,light snow,0.0,0.014173
318,Pevek,RU,2020-04-02 19:51:06,69.70,170.31,4.33,94,100,7.16,light snow,0.0,0.009449
328,Puksoozero,RU,2020-04-02 19:51:07,62.59,40.61,30.16,97,91,10.69,light snow,0.0,0.018898
344,Ayan,RU,2020-04-02 19:51:09,56.45,138.17,23.05,94,100,14.74,light snow,0.0,0.013386
357,Xining,CN,2020-04-02 19:51:11,36.62,101.77,30.43,38,100,3.18,light snow,0.0,0.006693
450,Chumikan,RU,2020-04-02 19:49:19,54.72,135.31,23.49,91,100,11.45,light snow,0.0,0.029528


In [31]:
cities_rain_snow_df = pd.merge(cities_rain_df, cities_snow_df, how='outer')
cities_rain_snow_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,Bolobo,CD,2020-04-02 19:49:48,-2.17,16.23,77.72,77,54,4.99,light rain,0.054331,0.000000
1,Aitape,PG,2020-04-02 19:48:35,-3.14,142.35,79.07,85,94,14.20,light rain,0.081496,0.000000
2,Cidreira,BR,2020-04-02 19:49:20,-30.18,-50.21,73.31,63,33,20.60,light rain,0.011417,0.000000
3,Rikitea,PF,2020-04-02 19:48:20,-23.12,-134.97,77.05,73,85,6.73,light rain,0.027165,0.000000
4,Kavieng,PG,2020-04-02 19:48:56,-2.57,150.80,82.83,75,15,5.41,light rain,0.012598,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
81,Pevek,RU,2020-04-02 19:51:06,69.70,170.31,4.33,94,100,7.16,light snow,0.000000,0.009449
82,Puksoozero,RU,2020-04-02 19:51:07,62.59,40.61,30.16,97,91,10.69,light snow,0.000000,0.018898
83,Ayan,RU,2020-04-02 19:51:09,56.45,138.17,23.05,94,100,14.74,light snow,0.000000,0.013386
84,Xining,CN,2020-04-02 19:51:11,36.62,101.77,30.43,38,100,3.18,light snow,0.000000,0.006693
